# Price Optimization

## Import Libraries

In [28]:
from pandas import DataFrame,Series,read_excel,read_csv,to_datetime,to_numeric,options
from numpy import ndarray,arange,argmax,inf
from matplotlib.pyplot import figure,axvline,xlabel,ylabel,title,legend,grid,show,ylim,xticks,xlim,plot,axhline,annotate
import matplotlib.pyplot as plt
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor,plot_tree
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from warnings import filterwarnings
from pyodbc import connect
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from IPython.display import clear_output
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.ticker as tick
from scipy.signal import find_peaks
filterwarnings('ignore')

In [2]:
options.display.float_format = "{:,.2f}".format

## Connect to SQL

In [3]:
server = 'MOE'
database = 'prod_WeisMarkets_RecoverNow'
connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Integrated Security={True};Autocommit={True};Trusted_Connection=yes;'
conn = connect(connectionString)
cursor_1 = conn.cursor()

## Supporting Functions

In [4]:
def get_data(columns:list,upc:str) -> DataFrame:
	cursor_1.execute(f"""
			DECLARE @upc BIGINT = {upc}
			DECLARE @top_rows INT = 100000

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202201] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202201] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202202] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202202] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202203] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202203] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202204] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202204] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202205] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202205] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202206] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202206] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202207] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202207] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202208] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202208] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202209] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202209] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202210] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202210] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202211] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202211] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202212] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202212] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202301] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202301] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202302] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202302] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202303] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202303] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202304] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202304] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202305] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202305] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202306] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202306] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202307] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202307] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202308] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202308] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202309] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202309] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202310] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202310] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202311] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202311] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202312] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202312] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202401] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202401] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202402] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202402] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202403] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202403] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202404] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202404] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202405] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202405] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202406] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202406] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc

			UNION ALL

			SELECT TOP(@top_rows) s.ScanDate,s.StoreNbr,s.UPCNbr,s.ScanQty,s.ScanAmt,s.Retail_ATG,m.CaseCost,m.CasePack,m.CaseCost/m.CasePack AS UnitVndrPrice
				FROM [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_MOVEMENT_202407] m
				JOIN [MOE].[prod_WeisMarkets_RecoverNow_Scan].[dbo].[ATG_SCAN_202407] s ON
				s.ClientFamilyID_ATG=m.ClientFamilyID_ATG
				AND s.BusinessUnit=m.BusinessUnit
				AND s.StoreNbr=m.ShipTo_StoreNbr
				AND s.UPCNbr=m.UPCNbr
				WHERE
					s.UPCNbr=@upc
				ORDER BY
					s.Retail_ATG
	    """)
	data:list = cursor_1.fetchall()
	data_list:list = []
	for index in range(len(data)):
		data_list.append(list(data[index]))
	del data
	return DataFrame(data=data_list,columns=columns)

In [5]:
def optimize_price(model:LinearRegression, X_sample:DataFrame, df_original:DataFrame, price_range:ndarray) -> tuple[float,float]:
    best_price = None
    best_margin = -inf
    
    for price in price_range:
        #X_sample_copy['Retail_ATG'] = price
        
        # Get the corresponding row from the original dataframe
        original_row = df_original.loc[X_sample.name]

        predicted_sales = model.predict([X_sample])[0]
        margin = (X_sample['Retail_ATG'] - original_row['UnitVndrPrice']) * predicted_sales
        
        if margin > best_margin:
            best_margin = margin
            best_price = price
            
    return best_price, best_margin

## Global Variables

In [6]:
random_state:int = 100
upc:str = '3400029005'

In [7]:
columns:list = [
		"ScanDate",
        "StoreNbr",
		"UPCNbr",
		"ScanQty",
		"ScanAmt",
		"Retail_ATG",
        "CaseCost",
        "CasePack",
        "UnitVndrPrice"
    ]

## Loading/Cleaning Data

In [8]:
df:DataFrame = get_data(columns=columns,upc=upc)

In [9]:
df_original = df.copy()

In [10]:
df['Year'] = df['ScanDate'].dt.year
df['Month'] = df['ScanDate'].dt.month
df['Day'] = df['ScanDate'].dt.day_of_week
df = df.drop(['ScanDate'], axis=1)
df = df.dropna()

In [11]:
df['ScanQty'] = df['ScanQty'].astype(float)
df['ScanAmt'] = df['ScanAmt'].astype(float)
df['Retail_ATG'] = df['Retail_ATG'].astype(float)
df['CaseCost'] = df['CaseCost'].astype(float)
df['CasePack'] = df['CasePack'].astype(float)
df['UnitVndrPrice'] = df['UnitVndrPrice'].astype(float)

In [12]:
df = df.sort_values(by=['Retail_ATG','Year','Month','Day'])

In [13]:
df['Margin_Unit'] = df['Retail_ATG'] - df['UnitVndrPrice']

In [14]:
df['Margin_Total'] = (df['Retail_ATG'] - df['UnitVndrPrice'])*(df['ScanQty'])

In [15]:
df['Cumulative_Margin_Total'] = df.groupby('Retail_ATG')['Margin_Total'].cumsum()

In [16]:
df['Cumulative_Sales_By_Price'] = df.groupby('Retail_ATG')['ScanAmt'].cumsum()

In [17]:
total_sales_by_price = df.groupby('Retail_ATG')['ScanAmt'].sum().reset_index()
total_sales_by_price.columns = ['Retail_ATG', 'Total_Sales_By_Price']

# Merge this total sales data back to the original DataFrame
df = df.merge(total_sales_by_price, on='Retail_ATG', how='left')

In [18]:
df['Cumulative_Total_Sales'] = df['ScanAmt'].cumsum()

In [19]:
df['Cumulative_Margin_Percent'] = df['Cumulative_Margin_Total'] / df['Cumulative_Sales_By_Price'] * 100

## Price Optimizing Using ML

In [20]:
X:DataFrame = df[['Retail_ATG','UnitVndrPrice']]#df.drop(['Retail_ATG'], axis=1)
y:Series = df['ScanQty']

In [34]:
X:DataFrame = df[['Retail_ATG']]#df.drop(['Retail_ATG'], axis=1)
y:Series = df['ScanQty']

In [ ]:
lr:LinearRegression = LinearRegression()
lr.fit(X, y)
dt:DecisionTreeRegressor = DecisionTreeRegressor(max_depth=2)
dt.fit(X,y)
xgb:XGBRegressor = XGBRegressor()
xgb.fit(X,y)
cb:CatBoostRegressor = CatBoostRegressor()
cb.fit(X,y)
knn:KNeighborsRegressor = KNeighborsRegressor()
knn.fit(X,y)

## Creating Optimal Price Spreadsheet

In [22]:
price_range:ndarray = arange(0, 7, 0.01)
predictions:list = []
for price in price_range:
    current_predictions_ai:list = []
    for model in [lr,dt,xgb,cb,knn]:
        current_predictions_ai.append(model.predict(DataFrame([[price,X['UnitVndrPrice'].quantile(0.9)]],columns=['Retail_ATG','UnitVndrPrice'])))
    predictions.append([price,min(current_predictions_ai),(price-X['UnitVndrPrice'].quantile(0.9))*min(current_predictions_ai)])

In [ ]:
plot_tree(dt)

In [ ]:
lr.coef_

In [ ]:
safe_sparse_dot(X, lr.coef_,dense_output=True) + lr.intercept_

In [ ]:
predicted_individual_sales_profits:DataFrame = DataFrame(columns=['Retail_Price','Vendor_Price','Predicted_Qty_Per_Transaction','Predicted_Individual_Profit'])

In [ ]:
for item in predictions:
    predicted_individual_sales_profits.loc[len(predicted_individual_sales_profits)] = [item[0],X['UnitVndrPrice'].quantile(0.9),item[1][0],item[2][0]]

In [ ]:
predicted_individual_sales_profits:DataFrame = predicted_individual_sales_profits[
    (predicted_individual_sales_profits['Predicted_Qty_Per_Transaction']>0)&
    (predicted_individual_sales_profits['Retail_Price']>X['UnitVndrPrice'].quantile(0.9))&
    (predicted_individual_sales_profits['Predicted_Individual_Profit']>0)]

In [ ]:
predicted_individual_sales_profits.insert(0,'Current_Retail_Price',float(df_original['Retail_ATG'].mean()))
predicted_individual_sales_profits.insert(1,'Current_Avg_Quantity',float(df_original['ScanQty'].sum()/len(df_original)))
predicted_individual_sales_profits.insert(2,'Current_Daily_Quantity',float(df_original['ScanQty'].sum()/df_original['ScanDate'].nunique()))

In [ ]:
predicted_individual_sales_profits['Predicted_Daily_Quantity'] = (predicted_individual_sales_profits['Predicted_Qty_Per_Transaction'])*(float(df_original['ScanQty'].count()/df_original['ScanDate'].nunique()))

In [ ]:
predicted_individual_sales_profits['Predicted_Daily_Profit'] = abs(predicted_individual_sales_profits['Predicted_Individual_Profit'])*predicted_individual_sales_profits['Predicted_Daily_Quantity']

In [ ]:
predicted_individual_sales_profits['Predicted_Yearly_Quantity'] = predicted_individual_sales_profits['Predicted_Daily_Quantity']*365

In [ ]:
predicted_individual_sales_profits['Predicted_Yearly_Profit'] = predicted_individual_sales_profits['Predicted_Daily_Profit']*365

In [ ]:
predicted_individual_sales_profits[predicted_individual_sales_profits['Predicted_Yearly_Profit']>0].sort_values(by='Retail_Price').to_excel(f'Predicted_Individual_Profits_{upc}.xlsx',index=False)

In [ ]:
DataFrame([[str(df_original['UPCNbr'].values[0]),
           predicted_individual_sales_profits[predicted_individual_sales_profits['Predicted_Yearly_Profit']==predicted_individual_sales_profits['Predicted_Yearly_Profit'].max()]['Retail_Price'].values[0]]],
           columns=['UPCNbr','Optimal_Price'])

In [ ]:
# Plot the results
graph = figure(figsize=(10,6))
ax = graph.add_subplot(1,1,1)
ax.get_xaxis().set_major_formatter(tick.FuncFormatter(lambda x,p: format(float(x), ',.2f')))
ax.get_yaxis().set_major_formatter(tick.FuncFormatter(lambda x,p: format(float(x), ',.2f')))
ax.plot(predicted_individual_sales_profits['Retail_Price'], predicted_individual_sales_profits['Predicted_Yearly_Profit'], label='Revenue', color='green')
peaks, _ = find_peaks(predicted_individual_sales_profits['Predicted_Yearly_Profit'])
final_peaks:list = []
for peak in peaks:
    if(peak in [439,318,299,276,228,205]):
        final_peaks.append(peak)
peaks = final_peaks.copy()
plot(predicted_individual_sales_profits['Retail_Price'][peaks], predicted_individual_sales_profits['Predicted_Yearly_Profit'][peaks], 'ro')
axvline(predicted_individual_sales_profits[(predicted_individual_sales_profits['Predicted_Yearly_Profit']==predicted_individual_sales_profits['Predicted_Yearly_Profit'].max())]['Retail_Price'].values[0], 
        color='blue', linestyle='--', 
        label=f'Optimal Price = ${predicted_individual_sales_profits[(predicted_individual_sales_profits["Predicted_Yearly_Profit"]==predicted_individual_sales_profits["Predicted_Yearly_Profit"].max())]["Retail_Price"].values[0]:,.2f}')
axhline(predicted_individual_sales_profits['Predicted_Yearly_Profit'].max(),
        color='red',linestyle='--',
        label=f'Max Revenue = ${predicted_individual_sales_profits["Predicted_Yearly_Profit"].max():,.2f}')
ylim(0,predicted_individual_sales_profits["Predicted_Yearly_Profit"].max()*1.1)
xlim(0,predicted_individual_sales_profits['Retail_Price'].max()*1.1)
for peak in peaks:
    offset_x:int = 0
    offset_y:int = 0
    if(peak==439):
        offset_x:int = -50
        offset_y:int = 10
    elif(peak==318):
        offset_x:int = 40
        offset_y:int = 5
    elif(peak in [299,276,228,205]):
        offset_x:int = -40
        offset_y:int = 5
    annotate(f'({predicted_individual_sales_profits["Retail_Price"][peak]:.2f}, {predicted_individual_sales_profits["Predicted_Yearly_Profit"][peak]:.2f})', 
                 (predicted_individual_sales_profits['Retail_Price'][peak], predicted_individual_sales_profits['Predicted_Yearly_Profit'][peak]), 
                 textcoords="offset points", 
                 xytext=(offset_x,offset_y), 
                 ha='center')
xticks([round(x/100,2) for x in range(0,int((predicted_individual_sales_profits['Retail_Price'].max()*1.1)*100),50)])
xlabel('Price ($)')
ylabel('Revenue ($)')
title('Price Optimization')
legend()
grid(True)
show()

In [ ]:
# Plot the results
prediction_data = read_excel(f'Predicted_Individual_Profits_{upc}.xlsx').sort_values(by='Retail_Price')
graph = figure(figsize=(10,6))
ax = graph.add_subplot(1,1,1)
ax.get_xaxis().set_major_formatter(tick.FuncFormatter(lambda x,p: format(float(x), ',.2f')))
ax.get_yaxis().set_major_formatter(tick.FuncFormatter(lambda x,p: format(float(x), ',.2f')))
ax.plot(prediction_data['Retail_Price'], prediction_data['Predicted_Yearly_Profit'], label='Profit', color='green')
peaks, _ = find_peaks(prediction_data['Predicted_Yearly_Profit'])
final_peaks:list = []
for peak in peaks:
    if(peak in [426,318,299,276,228,205]):
        final_peaks.append(peak)
peaks = final_peaks.copy()
plot(prediction_data['Retail_Price'][peaks], prediction_data['Predicted_Yearly_Profit'][peaks], 'ro')
axvline(prediction_data[(prediction_data['Predicted_Yearly_Profit']==prediction_data['Predicted_Yearly_Profit'].max())]['Retail_Price'].values[0], 
        color='blue', linestyle='--', 
        label=f'Optimal Price = ${prediction_data[(prediction_data["Predicted_Yearly_Profit"]==prediction_data["Predicted_Yearly_Profit"].max())]["Retail_Price"].values[0]:,.2f}')
axhline(prediction_data['Predicted_Yearly_Profit'].max(),
        color='red',linestyle='--',
        label=f'Max Profit = ${prediction_data["Predicted_Yearly_Profit"].max():,.2f}')
ylim(0,prediction_data["Predicted_Yearly_Profit"].max()*1.1)
xlim(0,prediction_data['Retail_Price'].max()*1.1)
for peak in peaks:
    offset_x:int = 0
    offset_y:int = 0
    if(peak==426):
        offset_x:int = 20
        offset_y:int = -16
    elif(peak==318):
        offset_x:int = 40
        offset_y:int = 5
    elif(peak in [299,276,228,205]):
        offset_x:int = -40
        offset_y:int = 5
    annotate(f'({prediction_data["Retail_Price"][peak]:.2f}, {prediction_data["Predicted_Yearly_Profit"][peak]:,.0f})', 
                 (prediction_data['Retail_Price'][peak], prediction_data['Predicted_Yearly_Profit'][peak]), 
                 textcoords="offset points", 
                 xytext=(offset_x,offset_y), 
                 ha='center')
xticks([round(x/100,2) for x in range(0,int((prediction_data['Retail_Price'].max()*1.1)*100),50)])
xlabel('Price ($)')
ylabel('Profit ($)')
title('Price Optimization')
legend()
grid(True)
show()

## Price Optimizing From Sums

In [ ]:
df_sums:DataFrame = df.groupby('Retail_ATG').sum().reset_index(drop=False)[['Retail_ATG','ScanQty','ScanAmt','Margin_Total']]

In [ ]:
X:DataFrame = df_sums[['Retail_ATG']]
y:Series = df_sums['ScanQty']

In [ ]:
model.fit(X,y)

In [ ]:
price_range:ndarray = arange(0, 7, 0.01)
predictions:list = []
for price in price_range:
    prediction = model.predict(DataFrame([[price]]))
    predictions.append([price,prediction,(price-df['UnitVndrPrice'].max())*prediction])
predicted_individual_sales_profits:DataFrame = DataFrame(columns=['Retail_Price','Vendor_Price','Predicted_Qty','Predicted_Profit'])
for item in predictions:
    predicted_individual_sales_profits.loc[len(predicted_individual_sales_profits)] = [item[0],df['UnitVndrPrice'].max(),item[1][0],item[2][0]]
predicted_individual_sales_profits.to_csv(f'Predicted_Total_Profits_{upc}.csv',index=False)

In [ ]:
# Predict sales at different prices
prices:ndarray = arange(0, df_sums['Retail_ATG'].max(), 0.01).reshape(-1, 1)  
predicted_sales:ndarray = model.predict(prices)

# Calculate revenue at each price point
revenue = prices.flatten() * predicted_sales

# Find the optimal price (price that maximizes revenue)
optimal_price = prices[argmax(revenue)][0]
max_revenue = max(revenue)

In [ ]:
for rev in range(len(revenue)):
    print(f"{prices[rev]} : {revenue[rev]:,.2f}")

In [ ]:
# Plot the results
graph = figure(figsize=(10,6))
ax = graph.add_subplot(1,1,1)
ax.get_xaxis().set_major_formatter(tick.FuncFormatter(lambda x,p: format(float(x), ',.2f')))
ax.get_yaxis().set_major_formatter(tick.FuncFormatter(lambda x,p: format(float(x), ',.2f')))
ax.plot(prices, revenue, label='Revenue', color='green')
axvline(optimal_price, color='red', linestyle='--', label=f'Optimal Price = ${optimal_price:,.2f}')
axhline(max_revenue,color='red',linestyle='--',label=f'Max Revenue = ${max_revenue:,.2f}')
ylim(0,62_000)
xlim(0,6)
xticks([round(x/100,2) for x in range(0,601,50)])
xlabel('Price ($)')
ylabel('Revenue ($)')
title('Price Optimization')
legend()
grid(True)
show()